In [2]:
#import all of the necessary packages for muliple classifiers
import pandas as pd
import numpy as np
import numpy
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LinearRegression, ARDRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm.classes import OneClassSVM
from sklearn.neighbors.classification import RadiusNeighborsClassifier
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.linear_model.ridge import RidgeClassifierCV
from sklearn.linear_model.ridge import RidgeClassifier
from sklearn.linear_model.passive_aggressive import PassiveAggressiveClassifier    
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB  
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder
from sklearn.base import ClassifierMixin
from sklearn.utils.testing import all_estimators
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import xgboost as xgb
# Fixing random state for reproducibility
np.random.seed(19680801)

In [10]:
#specify all the files we want to process in a list
all_genes = ["all_fly_genes.csv", "all_human_genes.csv"]
forty_one_genes = ["all_human_fly_overlap.csv", "all_fly_human_overlap.csv"]
aging_cor_genes = ["flies_aging_top_1000.csv", "human_aging_correlated_top_1000.csv" ]
homologous_genes = ["Transposed_back_othologous_genes_combined_species_zeros_dropped_fly.csv", "Transposed_back_othologous_genes_combined_species_zeros_dropped_human.csv"]
overlapping_genes = ['flies_257_human_aging_correlated_genes.csv', 'all_human_fly_correlations.csv']
# human_aging_correlated_top_1000 --- human top 1000 aging correlated genes
# all_human_fly_correlations.csv --- huamn top 800 something genes based on fly aging correlations
# all_human_fly_overlap --- human top 41 overlapping genes
# all_fly_human_overlap --- fly top 41 overlapping genes
# flies_aging_top_1000 --- all top 1000 fly aging genes
# flies_257_human_aging_correlated_genes.csv --- flies 257 human aging correlated genes overlap
# all_fly_genes --- all fly genes
#all_human_genes --- all_human_genes
# homologs fly genes... --- transposed_back...fly
# homologs. human genes.. --- transposed_back...human

lists_of_data = [all_genes, forty_one_genes, aging_cor_genes, homologous_genes, overlapping_genes]

In [ ]:
#create correlations of a dataset to select which correlated features we want to use
our_human_ = pd.read_csv("all_human_genes.csv")
# drop these columns from the dataframe
X_our_human_ = our_human_.drop(columns=['Individual',"Species","Age","Group"])
y_our_human_ = our_human_['Age']

#make correlation matrix
cor = our_human_.corr()
cor_df = pd.DataFrame(cor)
cor_df['Age'].sort_values(ascending=False).to_csv("fly_aging_correlations.csv")
#Correlation with output variable
cor_target = abs(cor['Age'])
# print(cor['Group'])
#Selecting highly correlated features
relevant_features = cor_target.sort_values(ascending=False)
# print(len(relevant_features))
feature_list = pd.DataFrame(relevant_features)
feature_list_names = feature_list.index.values
relavent_genes_all_human_aging = our_human_[feature_list_names].drop(columns=['Age'])
relavent_genes_all_human_aging['Age'] = our_human_['Age']
relavent_genes_all_human_aging['Individual'] = our_human_['Individual']
relavent_genes_all_human_aging['Species'] = our_human_['Species']
relavent_genes_all_human_aging['Group'] = our_human_['Group']
relavent_genes_all_human_aging.to_csv("relavent_human_aging_correlated_genes.csv", index=False)
#show relavant gene data on top correlation features



In [4]:
#specify a list of all classification models
classifiers_to_test = [xgb.XGBClassifier(),ExtraTreeClassifier(), DecisionTreeClassifier(),\
                       RadiusNeighborsClassifier(), KNeighborsClassifier(),SGDClassifier(),\
                       RidgeClassifier(), PassiveAggressiveClassifier(), GaussianProcessClassifier(),\
                       AdaBoostClassifier(), GradientBoostingClassifier(), BaggingClassifier(),\
                       ExtraTreesClassifier(), RandomForestClassifier(), LogisticRegression()]

In [7]:
def perf_measure(y_actual, y_pred):
    class_id = set(y_actual).union(set(y_pred))
    TP = []
    FP = []
    TN = []
    FN = []

    for index ,_id in enumerate(class_id):
        TP.append(0)
        FP.append(0)
        TN.append(0)
        FN.append(0)
        for i in range(len(y_pred)):
            if y_actual[i] == y_pred[i] == _id:
                TP[index] += 1
            if y_pred[i] == _id and y_actual[i] != y_pred[i]:
                FP[index] += 1
            if y_actual[i] == y_pred[i] != _id:
                TN[index] += 1
            if y_pred[i] != _id and y_actual[i] != y_pred[i]:
                FN[index] += 1


    return class_id,TP, FP, TN, FN

In [8]:
#create individual datasets of each species
for dataset in lists_of_data:
    for file in dataset:
        for classification_model in classifiers_to_test:

            df2_test = pd.read_csv(file)


            #create individual datasets of each species
            #set our Y predictor
            y_strings = df2_test['Group']
            #encode it to integers for use in the classification
            y = y_strings

            # drop these columns from the dataframe
            X_ = df2_test.drop(columns=['Individual',"Species","Age","Group"])

            model_for_use = classification_model

            avg_acc = list()
            f1_list = list()
            avg_tp = list()
            avg_fp = list()
            avg_tn = list()
            avg_fn = list()

            n_iterations = 1000
            for x in range(n_iterations):
                print(x, end="\r")
                X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.25, random_state=x)
                model_for_use.fit(X_train, y_train)
                predicted_ages = model_for_use.predict(X_test)
                avg_acc.append(sklearn.metrics.accuracy_score(y_test, predicted_ages))

                f1_list.append(sklearn.metrics.f1_score(y_test, predicted_ages, average="macro"))
                class_id, tp, fp, tn, fn = perf_measure(list(y_test), list(predicted_ages))
                avg_tp.append(sum(tp))
                avg_fp.append(sum(fp))
                avg_tn.append(sum(tn))
                avg_fn.append(sum(fn))


            print("{} using {}".format(file, classification_model))
            print("\nThe average Accuracy was: {}".format(round(sum(avg_acc)/len(avg_acc), 2)))
            print("\nThe average true positive was: {}".format(round(sum(avg_tp)/len(avg_tp), 0)))
            print("\nThe average false positive was: {}".format(round(sum(avg_fp)/len(avg_fp), 0)))
            print("\nThe average true negative  was: {}".format(round(sum(avg_tn)/len(avg_tn), 0)))
            print("\nThe average false negative was: {}".format(round(sum(avg_fn)/len(avg_fn), 0)))


KeyboardInterrupt: 

Next lets make all of the confusion matrices


In [11]:
#create individual datasets of each species
for dataset in lists_of_data:
    for file in dataset:
        for classification_model in classifiers_to_test:

            df2_test = pd.read_csv(file)


            #create individual datasets of each species
            #set our Y predictor
            y_strings = df2_test['Group']
            #encode it to integers for use in the classification
            y = y_strings

            # drop these columns from the dataframe
            X_ = df2_test.drop(columns=['Individual',"Species","Age","Group"])

            model_for_use = classification_model

            avg_acc = list()
            f1_list = list()
            avg_tp = list()
            avg_fp = list()
            avg_tn = list()
            avg_fn = list()

            n_iterations = 1000
            for x in range(n_iterations):
                print(x, end="\r")
                X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.25, random_state=x)
                model_for_use.fit(X_train, y_train)
                predicted_ages = model_for_use.predict(X_test)
                mat = confusion_matrix(y_test, predicted_ages)
                avg_matts.append(mat)

            average_matrix = [np.rint(sum(x)/len(x)) for x in zip(*avg_matts)]
            for x in average_matrix:
                for i in x:
                    i = int(i)
            plt.suptitle('{} using {}'.format(file, classification_model), size=16)
            sns.heatmap(average_matrix, square=True, annot=True,  cbar=False,annot_kws={"size": 16},
                        xticklabels=["Old","Middle","Young"], yticklabels=["Old","Middle","Young"])
            plt.xlabel('Predicted Age', size=16)
            plt.ylabel('True Age', size=16)


KeyboardInterrupt: 